In [45]:
import csv
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import re
import pickle
import os

In [46]:
stopwords = set(stopwords.words('english'))

In [47]:
k_gram_index = {}
k = 3

In [48]:
def remove_stopwords(tokens):
    tokens_wo_stopwords = []
    for i in range(0,len(tokens)):
        if tokens[i].lower() not in stopwords:
            tokens_wo_stopwords.append(tokens[i].lower())
    return tokens_wo_stopwords

In [49]:
def save(inverted_index,filename):
    with open(filename + '.pkl','wb') as index:
        pickle.dump(inverted_index,index,pickle.HIGHEST_PROTOCOL)

In [50]:
def read():
    with open("inverted_index.pkl",'rb') as file:
        inverted_index = pickle.load(file)
    with open("bi_word_inverted_index.pkl",'rb') as file1:
        bi_word_inverted_index = pickle.load(file1)

In [51]:
def load_and_preprocess():
    for i in range(5,100,5):
        with open("data/data_split_" + str(i) + ".csv") as file:
            csv_reader = csv.reader(file,delimiter=',')
            flag = 0
            for row in csv_reader:
                row = re.sub(r'[^a-zA-Z]', ' ', str(row))
                tokens = word_tokenize(str(row))
                tokens = remove_stopwords(tokens)
                add_to_inverted_index(tokens,i)
    save(k_gram_index,"inverted_index")

In [52]:
def add_to_inverted_index(tokens,index):
    for i in range(0,len(tokens)):
        token = '$' + tokens[i] + '$'
        for j in range(0,len(token) - k):
            substr = token[j:j + k]
            if substr not in k_gram_index:
                k_gram_index[substr] = [tokens[i]]
            else:
                if tokens[i] not in k_gram_index[substr]:
                    k_gram_index[substr].append(tokens[i])

In [53]:
def get_correct_spelling(word):
    result = {}
    query_word = '$' + word + '$'
    for i in range(0,len(query_word) - k):
        substr = query_word[i:i + k]
        if substr in k_gram_index:
            appeared_in = k_gram_index[substr]
            for j in range(0,len(appeared_in)):
                if appeared_in[j] not in result:
                    result[appeared_in[j]] = 1
                else:
                    result[appeared_in[j]] += 1
    result = sorted(result.items(), key=lambda x: x[1],reverse=True)
    print('Correct spelling for ' + query_word + ":- ")
    for i in range(0,min(len(result),5)):
        print(result[i][0])

In [54]:
load_and_preprocess()

In [56]:
query = input("Enter query:- ")
isolated_words = query.split(' ')
for i in range(0,len(isolated_words)):
    get_correct_spelling(isolated_words[i])

Enter query:- conent
component
content
confident
confidential
consequential
